<a href="https://colab.research.google.com/github/guy998877/projects/blob/main/json_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Working With Real Data**

In [ ]:
import pandas as pd
import re
import json
from datetime import date
import calendar

## **PART A** - Blog Authorship Corpus

In [ ]:
# Installing the Kaggle package
!pip install kaggle 

!mkdir /root/.kaggle/ 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"guyzagorski","key":"fda16236602e3bfd67033eb42daaeeed"}


# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rtatman/blog-authorship-corpus

 97% 280M/290M [00:02<00:00, 123MB/s]
100% 290M/290M [00:02<00:00, 137MB/s]


In [ ]:
!unzip blog-authorship-corpus.zip

Archive:  blog-authorship-corpus.zip
  inflating: blogtext.csv            


In [ ]:
blog = pd.read_csv("blogtext.csv")

In [ ]:
blog.head(10)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of su...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country o..."
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Take a read of this news article ...
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a l...


In [ ]:
blog.describe()

,id,age
count,6.812840e+05,681284.000000
mean,2.397802e+06,23.932326
std,1.247723e+06,7.786009
min,5.114000e+03,13.000000
25%,1.239610e+06,17.000000
50%,2.607577e+06,24.000000
75%,3.525660e+06,26.000000
max,4.337650e+06,48.000000


### **ANSWER 1**

First, we will sort the data frame to word or numbers greater than 8 in text. 

In [ ]:
blog_copy = blog.copy()
r = re.compile("\w{8,}")
df_text_8 = blog_copy[blog_copy['text'].str.count(r"\w{8,}") > 0]
df_text_8

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...
...,...,...,...,...,...,...,...
681279,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, I could write some really ..."
681280,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, 'I have the second yeast i..."
681281,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, Your 'boyfriend' is fuckin..."
681282,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan: Just to clarify, I am as..."


After we will execute group by sign

In [ ]:
sign_count = df_text_8.groupby("sign",as_index=False)["id"].count()
sign_count = sign_count.rename(columns={'id': 'count'})
sign_count

,sign,count
0,Aquarius,45164
1,Aries,58222
2,Cancer,59393
3,Capricorn,44545
4,Gemini,46635
5,Leo,48763
6,Libra,55740
7,Pisces,49101
8,Sagittarius,45262
9,Scorpio,51312


Get the max value 

In [ ]:
sign_count[sign_count['count']==sign_count['count'].max()]

,sign,count
2,Cancer,59393


### **ANSWER 2**

In [ ]:
# Get the number of month 
def month_string_to_number(string):
    m = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
         'may':5,
         'jun':6,
         'jul':7,
         'aug':8,
         'sep':9,
         'oct':10,
         'nov':11,
         'dec':12
        }
    try:
      s = string.strip()[:3].lower()
      out = m[s]
      return out
    except:
      return 0

# replce date format to  dd/mm/yyyy
def replace_date(date_string):
  r = re.compile(r"(\d{1,2}),([A-Za-z]+),(\d{4})")
  if r.findall(date_string):
    if r.findall(date_string)[0]:
      day = r.findall(date_string)[0][0]
      mon = month_string_to_number(r.findall(date_string)[0][1])
      year = r.findall(date_string)[0][2]
      return f'{day}/{mon}/{year}'

# find the day of the week by given date
def find_day_of_week(dt):
  if replace_date(dt):
    dt = replace_date(dt).split('/') 
    if dt[0] != '' and dt[1] != '0' and dt[2] != '':
      day, month, year = (int(x) for x in dt)    
      ans = date(year, month, day)
      return ans.strftime("%A")
  return None

Add another column day to the table based on date

In [ ]:
blog_day = blog
blog_day['day'] = blog_day['date'].apply(find_day_of_week)
blog_day

,id,gender,age,topic,sign,date,text,day
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",Friday
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,Thursday
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,Wednesday
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,Wednesday
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,Friday
...,...,...,...,...,...,...,...,...
681279,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, I could write some really ...",Thursday
681280,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, 'I have the second yeast i...",Thursday
681281,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, Your 'boyfriend' is fuckin...",Thursday
681282,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan: Just to clarify, I am as...",Thursday


Group by day

In [ ]:
blog_day_week_all = blog_day.groupby("day",as_index=False)["id"].count()
blog_day_week_all = blog_day_week_all.rename(columns={'id': 'count'})
blog_day_week_all

,day,count
0,Friday,86359
1,Monday,115733
2,Saturday,64479
3,Sunday,94510
4,Thursday,105584
5,Tuesday,110561
6,Wednesday,100297


In [ ]:
blog_day_week_all[blog_day_week_all['count']==blog_day_week_all['count'].max()]

,day,count
1,Monday,115733


### **ANSWER 3**

In [ ]:
blog['email_count'] = blog['text'].str.count(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}")
blog.groupby(['topic']).sum()['email_count']

topic
Accounting                   34
Advertising                  36
Agriculture                   3
Architecture                 33
Arts                        265
Automotive                    5
Banking                      25
Biotech                      78
BusinessServices             73
Chemicals                    32
Communications-Media        327
Construction                 17
Consulting                   74
Education                   321
Engineering                  83
Environment                   2
Fashion                      19
Government                   41
HumanResources               26
Internet                    389
InvestmentBanking             0
Law                          49
LawEnforcement-Security      18
Manufacturing                 7
Maritime                      4
Marketing                    65
Military                     17
Museums-Libraries            33
Non-Profit                  468
Publishing                   84
RealEstate                   15
Re

### **ANSWER 4**

First of all , we will count how many emails per blog and create a new
column count_email.

In [ ]:
emails_per_blogger = blog.groupby(['id']).sum()['email_count']
max_emails = emails_per_blogger.max()
max_emails

337

In [ ]:
emails_per_blogger[emails_per_blogger == max_emails]

id
1926378    337
4075390    337
Name: email_count, dtype: int64

The bloggers with the most emails at all the posts are **1926378** and **4075390**

### **ANSWER 5**

Func that find the longest num by regex

In [ ]:
#Find the longest number
def find_longest_num(ls):
  max = -1
  for elem in ls:
    elem = elem.replace(',', '')
    if len(elem) > max:
      max = len(elem)
  return max

In [ ]:
bloger_topic = blog.copy()
r = re.compile(r"[\d,]+")
bloger_topic["max_num"]=bloger_topic["text"].apply(lambda s : find_longest_num(r.findall(s)))
bloger_topic

,id,gender,age,topic,sign,date,text,day,email_count,max_num
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",Friday,0,3
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,Thursday,0,-1
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,Wednesday,2,4
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,Wednesday,0,-1
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,Friday,0,2
...,...,...,...,...,...,...,...,...,...,...
681279,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, I could write some really ...",Thursday,0,0
681280,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, 'I have the second yeast i...",Thursday,0,4
681281,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, Your 'boyfriend' is fuckin...",Thursday,0,0
681282,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan: Just to clarify, I am as...",Thursday,0,0


In [ ]:
bloger_topic[bloger_topic['max_num']==bloger_topic['max_num'].max()]

,id,gender,age,topic,sign,date,text,day,email_count,max_num
594474,3301387,male,15,Student,Taurus,"11,May,2004",[Hi hi hi hi N hi. Good day. lalala...,Tuesday,0,193


The topic of the blog with longest number

In [ ]:
bloger_topic_name = bloger_topic[bloger_topic['max_num']==bloger_topic['max_num'].max()]
bloger_topic_name["topic"]

594474    Student
Name: topic, dtype: object

## **PART B** - UFO Sightings

In [ ]:
# There are some bad lines with 12 fields instead of 11 so eliminate them
ufo = pd.read_csv("complete.csv", error_bad_lines=False)

b'Skipping line 878: expected 11 fields, saw 12\nSkipping line 1713: expected 11 fields, saw 12\nSkipping line 1815: expected 11 fields, saw 12\nSkipping line 2858: expected 11 fields, saw 12\nSkipping line 3734: expected 11 fields, saw 12\nSkipping line 4756: expected 11 fields, saw 12\nSkipping line 5389: expected 11 fields, saw 12\nSkipping line 5423: expected 11 fields, saw 12\nSkipping line 5614: expected 11 fields, saw 12\nSkipping line 5849: expected 11 fields, saw 12\nSkipping line 6093: expected 11 fields, saw 12\nSkipping line 7516: expected 11 fields, saw 12\nSkipping line 7626: expected 11 fields, saw 12\nSkipping line 8893: expected 11 fields, saw 12\nSkipping line 9015: expected 11 fields, saw 12\nSkipping line 9571: expected 11 fields, saw 12\nSkipping line 9620: expected 11 fields, saw 12\nSkipping line 9751: expected 11 fields, saw 12\nSkipping line 10157: expected 11 fields, saw 12\nSkipping line 10427: expected 11 fields, saw 12\n'


In [ ]:
ufo.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611


In [ ]:
ufo.describe()

,duration (seconds),latitude,longitude
count,1.197400e+04,11974.000000,11974.000000
mean,1.138953e+04,37.492445,-85.656229
std,9.018794e+05,10.709242,39.107644
min,0.000000e+00,-46.163992,-169.541667
25%,1.200000e+01,34.048928,-110.925833
50%,1.200000e+02,39.042361,-87.713056
75%,6.000000e+02,42.326667,-78.767431
max,9.783600e+07,70.292489,178.017649


### **ANSWER 1**

In [ ]:
def getUfoSights(df, country, year):
  ct = df.loc[df["country"] == country]
  get_year = re.compile('\d+\/\d+\/(\d+)')
  return len(ct['date posted'].apply(lambda x: x if get_year.findall(x)[0] == str(year) else None).dropna().index)

### **ANSWER 2**

In [ ]:
def get_month_year(s, i):
  r = re.compile(r'(\d+)\/\d+\/(\d+)')
  if r.findall(str(s)):
    return r.findall(str(s))[0][i]
  return -1

In [ ]:
date_ufo = ufo.copy()
date_ufo['month'] = ufo['datetime'].apply(lambda s: get_month_year(s, 0))
date_ufo['year'] = ufo['datetime'].apply(lambda s: get_month_year(s, 1))

In [ ]:
date_ufo.groupby(['year']).count()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,month
year,,,,,,,,,,,,
-1,1,0,0,0,0,0,0,0,0,0,0,1
1906,1,1,0,0,1,1,1,1,1,1,1,1
1910,1,1,1,1,1,1,1,1,1,1,1,1
1917,1,1,0,0,1,1,0,1,1,1,1,1
1936,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2010,647,647,615,580,634,647,626,647,647,647,647,647
2011,847,847,811,759,832,847,822,847,847,847,847,847
2012,1038,1038,986,936,1024,1038,1019,1038,1038,1038,1038,1038


As seen, there are some missing columns. 

But it seems that the datetime column is always filled.

In [ ]:
count_by_year = date_ufo.groupby(['year']).count()['datetime']
max_reports = count_by_year.max()
max_reports

1085

The maximum reports on a specific year is **1085**

In [ ]:
max_year = count_by_year[count_by_year == max_reports].index[0]
max_year

'2013'

The most UFO Sightings were spotted at **2013**

In [ ]:
date_ufo[date_ufo['year'] == str(max_year)].groupby(['month']).count()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,year
month,,,,,,,,,,,,
1,19,19,17,15,18,19,17,19,19,19,19,19
10,778,778,750,698,772,778,762,778,778,778,778,778
11,288,288,274,265,284,288,283,288,288,288,288,288


In [ ]:
count_by_month = date_ufo[date_ufo['year'] == str(max_year)].groupby(['month']).count()['datetime']
max_reports = count_by_month.max()
max_reports

778

In [ ]:
max_month = count_by_month[count_by_month == max_reports].index[0]
max_month

'10'

The most UFO Sightings were spotted at **2012** were spotted at **October**

### **ANSWER 3**

In [ ]:
ufo.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611


In [ ]:
get_time = re.compile(r'\s(\d+:\d+)')
time_ufo = ufo.copy()
time_ufo['time'] = time_ufo['datetime'].apply(lambda x: get_time.findall(x)[0] if get_time.findall(x) else None)
time_ufo.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,time
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,20:30
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,21:00
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,17:00
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,21:00
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,20:00


In [ ]:
count_by_time = time_ufo.groupby(['time']).count()['datetime']
max_reports = count_by_time.max()
max_reports

632

The max reports for a specific hour is **632**

In [ ]:
count_by_time[count_by_time == max_reports]

time
20:00    632
Name: datetime, dtype: int64

The time where the most reports were received was **20:00**

### **ANSWER 4**

In [ ]:
r = re.compile(r'[A-Za-z]{7,}\s[A-Za-z]{7,}')
ufo['count_pairs'] = ufo['comments'].str.count(r'[A-Za-z]{7,}\s[A-Za-z]{7,}')
ufo['count_pairs'].max()

4.0

The maximum pairs of words with 7 or more letters is **4**

The comment that contains 4 pairs of words with 7 or more  letters is:

In [ ]:
comment = list(ufo[ufo['count_pairs'] == ufo['count_pairs'].max()]['comments'])[0]
comment

'Object was a birdlike formation suddenly apearing in the sky then gradually passing toward north-east and suddenly vanishing. Event las'

The pairs found to match the pattern in this comment are:

In [ ]:
re.compile(r'[A-Za-z]{7,}\s[A-Za-z]{7,}').findall(comment)

['birdlike formation',
 'suddenly apearing',
 'gradually passing',
 'suddenly vanishing']

### **ANSWER 5**

In [ ]:
max_big_nums = ufo['comments'].str.count(r'\b\d{6}\b').max()
max_big_nums

2.0

In [ ]:
comment = list(ufo[ufo['comments'].str.count(r'\b\d{6}\b') == max_big_nums]['comments'])[0]
comment

'i seen two ufos&#44one last sep&#442002  near crum wv&#44one  oct 24&#442003  on 64 going to huntington wv&#44they both was triangle shped'

In [ ]:
re.compile(r'\b\d{6}\b').findall(comment)

['442002', '442003']

In [ ]:
ufo[ufo['comments'] == comment]

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,count_pairs
4690,10/24/2003 21:00,huntington,wv,us,triangle,0.0,NaN,i seen two ufos&#44one last sep&#442002 near ...,10/31/2003,38.419167,-82.445278,0.0


The country is NaN at this case, but it is mentioned in the city. The country that has the most 6 digit numbers in the comments is **Russia**

### **ANSWER 6**

In [ ]:
pattern = r'[a-zA-Z0-9]{2}\d{1}[a-zA-Z0-9]{1}'
ufo[ufo['comments'].str.count(pattern) > 0]

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,count_pairs
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,0.0
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,0.0
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,0.0
8,10/10/1966 20:00,pell city,al,us,disk,180.0,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.586111,-86.286111,0.0
15,10/10/1972 19:00,harlan county,ky,us,circle,1200.0,20minutes,On october 10&#44 1972 myself&#44my 5yrs.daugh...,9/15/2005,36.843056,-83.321944,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11950,1/1/1968 01:00,udorn (thailand),NaN,NaN,sphere,1800.0,30 minute,Zig Zagging bright light over USAF Air Base in...,5/2/2014,17.364697,102.815892,0.0
11956,1/1/1969 23:00,nevada test site,nv,NaN,triangle,0.0,?,Response to: Occurred : 4/1/2001 21:00 (Entere...,4/28/2001,36.813842,-116.288378,0.0
11959,1/1/1970 12:00,san juan (puerto rico),NaN,NaN,disk,300.0,5 minutes,Circa 1970,4/18/2012,0.000000,0.000000,0.0
11963,1/1/1972 16:00,seattle,wa,us,triangle,30.0,30 sec.,Man reports sighting in l972 or 1973. Heard w...,11/2/1999,47.606389,-122.330833,1.0
